In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from math import sqrt
from math import acos
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
from immuno_ms2rescore_tools.file_utilities import PrositLib
import pickle



In [2]:
def ms2pip_pearson(true, pred):
    """
    Return pearson of tic-normalized, log-transformed intensities, 
    the MS2PIP way.
    """
    #tic_norm = lambda x: x / np.sum(x)
    # log_transform = lambda x: np.log2(x + 0.001)
    corr = pearsonr(
        true, 
        pred
    )[0]
    return (corr)

In [3]:
def spectral_angle(true, pred, epsilon=1e-7):
    """
    Return square root normalized spectral angle.
    See https://doi.org/10.1074/mcp.O113.036475
    """
    
    de_log = lambda x: (2**x)-0.001
    l2_normalize = lambda x: x / sqrt(max(sum(x**2), epsilon))
    
    pred_norm = l2_normalize(de_log(pred))
    true_norm = l2_normalize(de_log(true))
    
    spectral_angle = 1 - (2 * acos(np.dot(pred_norm, true_norm)) / np.pi)

    return (spectral_angle)

In [4]:
class Scorer:
    def __init__(self,psmids):
        self.psmids = psmids
        
    def psm_score(self,targets, predictions):
        tmp = pd.DataFrame(columns=["psmids", "targets", "predictions"])
        tmp["psmids"] = np.array(self.psmids)
        tmp["targets"] = np.array(targets)
        tmp["predictions"] = np.array(predictions)
        tmp2 = tmp.groupby("psmids").agg({'predictions': list, 'targets': list}).reset_index()
        spectral_corr = []
        pearson_corr = []
        for spectra in range(0, len(tmp2["psmids"])):
            spectral_corr.append(spectral_angle(np.array(tmp2.targets.loc[spectra]), np.array(tmp2.predictions.loc[spectra])))
            pearson_corr.append(ms2pip_pearson(np.array(tmp2.targets.loc[spectra]), np.array(tmp2.predictions.loc[spectra])))
        return (pearson_corr, spectral_corr, tmp2["psmids"]) 

In [5]:
myprositlib_pxd005231 = PrositLib("data/evaluation_data/PXD005231/myPrositLib_PXD005231.csv")
myprositlib_pxd005231.remove_carbamidomethyl()
myprositlib_pxd005231.merge_spec_ids("data/evaluation_data/PXD005231/PXD005231_prosit.csv")
prosit_immunopeptide = myprositlib_pxd005231.create_pred_and_emp_csv("data/evaluation_data/PXD005231/spec_lib_PXD005231_HCD_pred_and_emp.csv")


In [6]:
prosit_test = pd.DataFrame(columns = ["pearson correlation", "spectral angle"])
prosit_scorer = Scorer(prosit_immunopeptide["spec_id"])
prosit_test["pearson correlation"], prosit_test["spectral angle"],prosit_test["psmid"]  = prosit_scorer.psm_score(prosit_immunopeptide["target"],prosit_immunopeptide["prediction"])
prosit_test["evaluation data set"] = "shotgun proteomics"

/home/arthur/miniconda3/envs/ms2pip-dev/lib/python3.8/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [7]:
#prosit_immunopeptide["spec_id"] == "mzspec:PXD005231:20160902_QEh1_LC2_CHC_SA_HLApI_TIL3_2:scan:49077"

In [8]:
myprositlib_pxd020011 = PrositLib("data/evaluation_data/PXD020011/myPrositLib_PXD020011.csv")
myprositlib_pxd020011.remove_carbamidomethyl()
myprositlib_pxd020011.merge_spec_ids("data/evaluation_data/PXD020011/PXD020011_prosit.csv")
prosit_hlaII = myprositlib_pxd020011.create_pred_and_emp_csv("data/evaluation_data/PXD020011/spec_lib_pxd020011_HCD_pred_and_emp.csv")
prosit_hlaII_Y = prosit_hlaII[prosit_hlaII["ion"] == "Y"]
prosit_hlaII_B = prosit_hlaII[prosit_hlaII["ion"] == "B"]

In [9]:
myprositlib_pxd020011.prositlib[myprositlib_pxd020011.prositlib["spec_id"] == "mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_Leuka2_MDC_2_R2:scan:5469"]

,RelativeIntensity,FragmentMz,ModifiedPeptide,LabeledPeptide,StrippedPeptide,PrecursorCharge,PrecursorMz,iRT,proteotypicity,FragmentNumber,FragmentType,FragmentCharge,FragmentLossType,spec_id
142506,0.700359,156.076752,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,4,544.541711,-13.758358,0.298364,1,y,1,noloss,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...
142507,0.013189,285.119354,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,4,544.541711,-13.758358,0.298364,2,y,1,noloss,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...
142508,0.000189,89.707779,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,4,544.541711,-13.758358,0.298364,2,b,3,noloss,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...
142509,0.000716,398.203400,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,4,544.541711,-13.758358,0.298364,3,y,1,noloss,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...
142510,0.003058,141.741486,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,4,544.541711,-13.758358,0.298364,3,b,3,noloss,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...
142511,0.047132,554.304504,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,4,544.541711,-13.758358,0.298364,4,y,1,noloss,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...
142512,0.015999,277.655884,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,4,544.541711,-13.758358,0.298364,4,y,2,noloss,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...
142513,0.000175,170.752151,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,4,544.541711,-13.758358,0.298364,4,b,3,noloss,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...
142514,0.112026,683.347107,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,4,544.541711,-13.758358,0.298364,5,y,1,noloss,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...
142515,0.056192,342.177185,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,EHRSRLEKSLQKERLEH,4,544.541711,-13.758358,0.298364,5,y,2,noloss,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...


In [10]:
prosit_hlaII[prosit_hlaII["spec_id"] == "mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_Leuka2_MDC_2_R2:scan:5469"]

,spec_id,prediction,FragmentMz,charge,ionnumber,ion,mz,target
236470,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...,-9.965784,130.049820,4,1,B,130.04982,-9.965784
236471,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...,-9.716269,89.707779,4,2,B,267.10873,-8.430389
236472,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...,-7.945179,141.741486,4,3,B,423.20984,-9.965784
236473,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...,-9.733220,170.752151,4,4,B,510.24185,-9.965784
236474,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...,-6.751124,333.675140,4,5,B,666.34296,-9.965784
236475,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...,-7.351073,390.217194,4,6,B,779.42700,-9.965784
236476,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...,-5.066518,454.738495,4,7,B,908.46960,-9.965784
236477,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...,-4.099890,518.785950,4,8,B,1036.56450,-9.965784
236478,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...,-8.846941,346.193054,4,8,B,1036.56450,-9.965784
236479,mzspec:True:20171216_QEh1_LC1_HLAIIp_SA_FaMa_L...,-4.328706,562.302002,4,9,B,1123.59640,-9.965784


In [4]:
chunks = []
chunksize = 10 ** 6
with pd.read_table("data/PXD021398/msms_full.txt", chunksize=chunksize) as reader:
    for parts in reader:
        chunks.append(parts)
msms = pd.concat(chunks)


/home/arthur/miniconda3/envs/ms2pip-dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3364: DtypeWarning: Columns (8,9,56) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/arthur/miniconda3/envs/ms2pip-dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3364: DtypeWarning: Columns (56) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/arthur/miniconda3/envs/ms2pip-dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3364: DtypeWarning: Columns (8,9,11,56) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
msms.head()

,Raw file,Scan number,Scan index,Sequence,Length,Missed cleavages,Modifications,Modified sequence,Oxidation (M) Probabilities,Oxidation (M) Score Diffs,...,Reverse,All scores,All sequences,All modified sequences,id,Protein group IDs,Peptide ID,Mod. peptide ID,Evidence ID,Oxidation (M) site IDs
0,C20150414_JGA_HLA_B57_star_3uLof6uL_5e7ceqonco...,19535,10232,AAAAAAAAAAAAAAAAAAYYQ,21,NaN,Unmodified,_AAAAAAAAAAAAAAAAAAYYQ_,NaN,NaN,...,NaN,8.284077;7.582434;5.541086,AAAAAAAAAAAAAAAAAAYYQ;AAAAAAAAAANGPAPTAAANAS;C...,_AAAAAAAAAAAAAAAAAAYYQ_;_AAAAAAAAAANGPAPTAAANA...,0,71278,0,0,0,NaN
1,M20150824_JGA_HLA_B4402_biorep1_5e7ceq_AcOHmob...,18925,17464,AAAAAAAAAAGGLAAA,16,NaN,Unmodified,_AAAAAAAAAAGGLAAA_,NaN,NaN,...,NaN,2.221972;2.154223;2.154223,AAAAAAAAAAGGLAAA;NVEKYRVVY;QHRQFQARV,_AAAAAAAAAAGGLAAA_;_NVEKYRVVY_;_QHRQFQARV_,1,65369,1,1,1,NaN
2,M20150626_JGA_HLA_B57_biorep2_5e7ceq_reinject_...,21933,17901,AAAAAAAAAW,10,NaN,Unmodified,_AAAAAAAAAW_,NaN,NaN,...,NaN,42.59861;37.55836;24.0301,AAAAAAAAAW;VNNVAAAW;GVQAAAAWA,_AAAAAAAAAW_;_VNNVAAAW_;_GVQAAAAWA_,2,70356,2,2,2,NaN
3,C20150414_JGA_HLA_B57_3uLof6uL_5e7ceqoncolumn_...,22498,11854,AAAAAAAAAW,10,NaN,Unmodified,_AAAAAAAAAW_,NaN,NaN,...,NaN,24.7756;20.52031;17.23078,AAAAAAAAAW;VNNVAAAW;GVQAAAAWA,_AAAAAAAAAW_;_VNNVAAAW_;_GVQAAAAWA_,3,70356,2,2,3,NaN
4,C20150414_JGA_HLA_B57_3uLof6uL_5e7ceqoncolumn_...,22595,11943,AAAAAAAAAW,10,NaN,Unmodified,_AAAAAAAAAW_,NaN,NaN,...,NaN,33.26252;29.37615;19.7865,AAAAAAAAAW;VNNVAAAW;AAAAAAVHY,_AAAAAAAAAW_;_VNNVAAAW_;_AAAAAAVHY_,4,70356,2,2,3,NaN


In [34]:
xl_file = pd.ExcelFile("/home/compomics/conode55_pride/Arthur/immunopeptidomics_PRIDE/PXD021398/NIHMS1541512-supplement-Sup_Data1.xlsx")

In [36]:
hla_alleles = xl_file.sheet_names[1:]

In [44]:
hla_raw_dict = {}
for hla_type in hla_alleles:
    hla_df = xl_file.parse(hla_type)
    hla_raw_dict[hla_type] = set(hla_df["filename"].str.split(".", 1, expand=True)[0])

In [48]:
raw_hla_map = {}
for hla, raw_file_list in hla_raw_dict.items():
    for rawfile in raw_file_list:
        raw_hla_map[rawfile] = hla

In [51]:
with open("data/PXD021398/HLA_allele_rawfile_mapping.pkl", "wb") as fp:
    pickle.dump(raw_hla_map, fp)